# Kelompok 9 Big Data Kelas B
- Muhammad Arkananta Radithya T - 5027221003
- Nicholas Marco Weinandra - 5027221042
- Sighra Attariq Sumere Jati - 5027221068

## Import PySpark

In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
print(sys.executable)

C:\Users\LOQ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe


In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
print ("Done")

Done


## Inisialisasi Spark Session

In [4]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.master("local").appName("Tugas").getOrCreate()

spark


## Load Dataset dengan Spark

In [5]:
df = spark.read.csv('C:/Users/LOQ/Documents/SEMESTER 5/Big Data/Tugas PySpark/US_Accidents_March23.csv/US_Accidents_March23.csv', header=True, inferSchema=True)
df.show(5)

+---+-------+--------+-------------------+-------------------+-----------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Ameni

# Preprocessing

## Convert Data Waktu

In [6]:
from pyspark.sql.functions import col, round as pyspark_round, to_timestamp, year, month, dayofmonth, hour, date_format
from pyspark.sql import functions as F

# Misalkan 'df' adalah PySpark DataFrame
# Convert Start_Time and End_Time to timestamp
df = df.withColumn('Start_Time', to_timestamp(col('Start_Time')))
df = df.withColumn('End_Time', to_timestamp(col('End_Time')))

# Extract year, month, day, hour and weekday
df = df.withColumn('Year', year(col('Start_Time')))
df = df.withColumn('Month', date_format(col('Start_Time'), 'MMM'))
df = df.withColumn('Day', dayofmonth(col('Start_Time')))
df = df.withColumn('Hour', hour(col('Start_Time')))
df = df.withColumn('Weekday', date_format(col('Start_Time'), 'E'))

# Calculate the duration in minutes (round to nearest integer)
df = df.withColumn('Time_Duration(min)', 
                   pyspark_round((F.col('End_Time').cast("long") - F.col('Start_Time').cast("long")) / 60))

# Show the schema of the DataFrame
df.printSchema()

# Display first few rows (optional)
df.show()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

In [7]:
# Print all column names in the DataFrame
print(df.columns)

['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Year', 'Month', 'Day', 'Hour', 'Weekday', 'Time_Duration(min)']


## Feature Selection

In [8]:
# Set the list of features to include in Machine Learning
feature_lst=['Source','Severity','Start_Lng','Start_Lat','Distance(mi)','City','State','Timezone', 'Visibility(mi)', 'Wind_Direction','Weather_Condition','Amenity','Bump','Crossing','Junction','Stop','Traffic_Calming','Traffic_Signal','Hour','Weekday', 'Time_Duration(min)']

In [9]:
from pyspark.sql import functions as F
    
# Select the dataset to include only the selected features
df_sel = df.select([F.col(c) for c in feature_lst])

# Show schema of the selected DataFrame
df_sel.printSchema()

# Show some rows (optional)
df_sel.show()


root
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Visibility(mi): double (nullable = true)
 |-- Wind_Direction: string (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Amenity: boolean (nullable = true)
 |-- Bump: boolean (nullable = true)
 |-- Crossing: boolean (nullable = true)
 |-- Junction: boolean (nullable = true)
 |-- Stop: boolean (nullable = true)
 |-- Traffic_Calming: boolean (nullable = true)
 |-- Traffic_Signal: boolean (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Weekday: string (nullable = true)
 |-- Time_Duration(min): double (nullable = true)

+-------+--------+------------------+------------------+------------+------------+-----+----------+--------------+----

## Handle Missing Values

In [10]:
from pyspark.sql.functions import col, sum, when

# Count the number of missing values (null or NaN) in each column
missing_values = df_sel.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_sel.columns])

# Show the result
missing_values.show()


+------+--------+---------+---------+------------+----+-----+--------+--------------+--------------+-----------------+-------+----+--------+--------+----+---------------+--------------+----+-------+------------------+
|Source|Severity|Start_Lng|Start_Lat|Distance(mi)|City|State|Timezone|Visibility(mi)|Wind_Direction|Weather_Condition|Amenity|Bump|Crossing|Junction|Stop|Traffic_Calming|Traffic_Signal|Hour|Weekday|Time_Duration(min)|
+------+--------+---------+---------+------------+----+-----+--------+--------------+--------------+-----------------+-------+----+--------+--------+----+---------------+--------------+----+-------+------------------+
|     0|       0|        0|        0|           0| 253|    0|    7808|        177098|        175206|           173459|      0|   0|       0|       0|   0|              0|             0|   0|      0|                 0|
+------+--------+---------+---------+------------+----+-----+--------+--------------+--------------+-----------------+-------+--

In [11]:
# Drop rows with any missing values
df_sel = df_sel.dropna()

# Show the result to verify
df_sel.show()

+-------+--------+------------------+------------------+------------+------------+-----+----------+--------------+--------------+-----------------+-------+-----+--------+--------+-----+---------------+--------------+----+-------+------------------+
| Source|Severity|         Start_Lng|         Start_Lat|Distance(mi)|        City|State|  Timezone|Visibility(mi)|Wind_Direction|Weather_Condition|Amenity| Bump|Crossing|Junction| Stop|Traffic_Calming|Traffic_Signal|Hour|Weekday|Time_Duration(min)|
+-------+--------+------------------+------------------+------------+------------+-----+----------+--------------+--------------+-----------------+-------+-----+--------+--------+-----+---------------+--------------+----+-------+------------------+
|Source2|       3|        -84.058723|         39.865147|        0.01|      Dayton|   OH|US/Eastern|          10.0|          Calm|       Light Rain|  false|false|   false|   false|false|          false|         false|   5|    Mon|             314.0|
|Sou

In [12]:
df_sel.printSchema()

root
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Visibility(mi): double (nullable = true)
 |-- Wind_Direction: string (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Amenity: boolean (nullable = true)
 |-- Bump: boolean (nullable = true)
 |-- Crossing: boolean (nullable = true)
 |-- Junction: boolean (nullable = true)
 |-- Stop: boolean (nullable = true)
 |-- Traffic_Calming: boolean (nullable = true)
 |-- Traffic_Signal: boolean (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Weekday: string (nullable = true)
 |-- Time_Duration(min): double (nullable = true)



## Encode Data Kategorikal dengan OHE

In [13]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Identifikasi kolom kategorikal (tipe data string)
categorical_columns = ['Source', 'City', 'State', 'Timezone', 'Wind_Direction', 'Weather_Condition', 'Weekday']

# Buat daftar tahap dalam pipeline untuk indexing dan encoding
indexers = [StringIndexer(inputCol=column, outputCol=column + "_index", handleInvalid="skip") for column in categorical_columns]
encoders = [OneHotEncoder(inputCol=column + "_index", outputCol=column + "_ohe") for column in categorical_columns]

# Gabungkan indexer dan encoder dalam satu pipeline
pipeline = Pipeline(stages=indexers + encoders)

# Terapkan pipeline ke DataFrame
df_sel_encoded = pipeline.fit(df_sel).transform(df_sel)

# Tampilkan hasil DataFrame yang di-encode (semua kolom, termasuk kolom yang baru di-encode)
df_sel_encoded.select([col for col in df_sel_encoded.columns] + [col + "_ohe" for col in categorical_columns]).show(truncate=False)


+-------+--------+------------------+------------------+------------+------------+-----+----------+--------------+--------------+-----------------+-------+-----+--------+--------+-----+---------------+--------------+----+-------+------------------+------------+----------+-----------+--------------+--------------------+-----------------------+-------------+-------------+--------------------+---------------+-------------+------------------+---------------------+-------------+-------------+--------------------+---------------+-------------+------------------+---------------------+-------------+
|Source |Severity|Start_Lng         |Start_Lat         |Distance(mi)|City        |State|Timezone  |Visibility(mi)|Wind_Direction|Weather_Condition|Amenity|Bump |Crossing|Junction|Stop |Traffic_Calming|Traffic_Signal|Hour|Weekday|Time_Duration(min)|Source_index|City_index|State_index|Timezone_index|Wind_Direction_index|Weather_Condition_index|Weekday_index|Source_ohe   |City_ohe            |State_oh

## Menggabungkan Semua Fitur

In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Kolom fitur: Semua kolom yang telah di-encode dan fitur numerik lainnya
feature_columns = [col for col in df_sel_encoded.columns if col.endswith('_ohe') or col in ['Start_Lng', 'Start_Lat', 'Distance(mi)', 'Visibility(mi)', 'Hour', 'Time_Duration(min)']]

# Menggabungkan semua fitur menjadi satu vektor menggunakan VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transformasi DataFrame: Menggabungkan fitur
df_final = assembler.transform(df_sel_encoded)

# Hasil akhirnya adalah kolom `features` dan target `Severity`
df_final = df_final.select("features", "Severity")


In [15]:
# Tampilkan skema dari df_final untuk memastikan kolom `features` dan `Severity` ada
df_final.printSchema()

# Tampilkan beberapa baris untuk melihat apakah kolom 'features' sudah benar
df_final.show(5, truncate=False)

root
 |-- features: vector (nullable = true)
 |-- Severity: integer (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------+--------+
|features                                                                                                                                |Severity|
+----------------------------------------------------------------------------------------------------------------------------------------+--------+
|(13411,[0,1,2,3,4,5,7,48,13206,13237,13246,13269,13409],[-84.058723,39.865147,0.01,10.0,5.0,314.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])         |3       |
|(13411,[0,1,2,3,4,5,7,2014,13206,13237,13246,13269,13409],[-82.831184,39.92805900000001,0.01,10.0,6.0,30.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|2       |
|(13411,[0,1,2,3,4,5,7,557,13206,13237,13247,13268,13409],[-84.032608,39.063148,0.01,10.0,6.0,30.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])         |2       |
|(13411,[0,1,2,3,4,5,7,48

In [16]:
# Cek jumlah nilai null di setiap kolom fitur
df_sel_encoded.select(feature_columns).agg(*[F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in feature_columns]).show()

+---------+---------+------------+--------------+----+------------------+----------+--------+---------+------------+------------------+---------------------+-----------+
|Start_Lng|Start_Lat|Distance(mi)|Visibility(mi)|Hour|Time_Duration(min)|Source_ohe|City_ohe|State_ohe|Timezone_ohe|Wind_Direction_ohe|Weather_Condition_ohe|Weekday_ohe|
+---------+---------+------------+--------------+----+------------------+----------+--------+---------+------------+------------------+---------------------+-----------+
|        0|        0|           0|             0|   0|                 0|         0|       0|        0|           0|                 0|                    0|          0|
+---------+---------+------------+--------------+----+------------------+----------+--------+---------+------------+------------------+---------------------+-----------+



# Split Train Test

In [17]:
# Sampling data, misalnya gunakan hanya 50% dari data yang tersedia
df_sampled = df_final.sample(fraction=0.01, seed=42)

# Split data yang sudah disampling menjadi train dan test sets (misalnya 80% train dan 20% test)
train_data, test_data = df_sampled.randomSplit([0.8, 0.2], seed=42)

# Decision Tree

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Inisialisasi Decision Tree Classifier (menggunakan 'gini' sebagai kriteria default)
dt = DecisionTreeClassifier(labelCol="Severity", featuresCol="features", maxDepth=4)

# Latih model pada training data
dt_model = dt.fit(train_data)

# Prediksi pada test data
predictions = dt_model.transform(test_data)

# Tampilkan beberapa prediksi++ 
predictions.select("features", "Severity", "prediction").show(5)

+--------------------+--------+----------+
|            features|Severity|prediction|
+--------------------+--------+----------+
|(13411,[0,1,2,3,4...|       2|       2.0|
|(13411,[0,1,2,3,4...|       3|       2.0|
|(13411,[0,1,2,3,4...|       2|       2.0|
|(13411,[0,1,2,3,4...|       2|       2.0|
|(13411,[0,1,2,3,4...|       3|       2.0|
+--------------------+--------+----------+
only showing top 5 rows



In [19]:
# Inisialisasi evaluator untuk klasifikasi
evaluator = MulticlassClassificationEvaluator(labelCol="Severity", predictionCol="prediction", metricName="accuracy")

# Hitung akurasi model
accuracy = evaluator.evaluate(predictions)

[Decision Tree -- gini] Accuracy score: 0.802
[Random Forest] F1 score: 0.730
[Random Forest] Precision score: 0.747


In [21]:
# Tampilkan hasil akurasi
print("[Decision Tree -- gini] Accuracy score: {:.3f}".format(accuracy))

# Hitung F1 score
f1_rf = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print("[Decision Tree] F1 score: {:.3f}".format(f1_rf))

# Hitung Precision
precision_rf = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
print("[Decision Tree] Precision score: {:.3f}".format(precision_rf))

[Decision Tree -- gini] Accuracy score: 0.802
[Decision Tree] F1 score: 0.730
[Decision Tree] Precision score: 0.747


In [20]:
# Pastikan Decision Tree telah dilatih sebelumnya
# Misalnya: dt_model = dt.fit(train_data)

# Ambil feature importance dari model Decision Tree
feature_importances = dt_model.featureImportances

# Gabungkan nama fitur dan importance dalam satu list
feature_importance_list = [(col, feature_importances[i]) for i, col in enumerate(feature_columns)]

# Urutkan berdasarkan nilai importance (dari tertinggi ke terendah)
feature_importance_sorted = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)

# Tampilkan fitur yang diurutkan berdasarkan importance
print("Feature Importances (Sorted):")
for feature, importance in feature_importance_sorted:
    print(f"{feature}: {importance}")


Feature Importances (Sorted):
Source_ohe: 0.7530849728623232
Time_Duration(min): 0.08470811022794388
Distance(mi): 0.07805722069589045
Hour: 0.043650897043404614
Start_Lng: 0.0
Start_Lat: 0.0
Visibility(mi): 0.0
City_ohe: 0.0
State_ohe: 0.0
Timezone_ohe: 0.0
Wind_Direction_ohe: 0.0
Weather_Condition_ohe: 0.0
Weekday_ohe: 0.0
